In [1]:
# Python reference for a Unified Random Sampler (Uniform, Ternary, Discrete Gaussian via CDT)
# Author: Muhammad Ogin Hasanuddin
# Requirements: numpy

import numpy as np

In [2]:
qp = np.array([281474976317441, 140737518764033, 140737470791681, 140737513783297,
        140737471578113, 140737513259009, 140737471971329, 140737509851137,
        140737480359937, 140737509457921, 140737481801729, 140737508671489,
        140737482981377, 140737506705409, 140737483898881, 140737504608257,
        140737484685313, 140737499496449, 140737485864961, 140737493729281,
        140737486520321, 140737490976769, 140737487306753, 140737488486401,
        281474975662081, 281474974482433, 281474966880257, 281474962554881,
        281474960326657, 281474957180929, 281474955476993, 281474952462337],
       dtype=object)

In [11]:
# Reference: https://ieeexplore.ieee.org/abstract/document/11043509
# ------------------------------
# Helpers (Barrett reduce, etc.)
# ------------------------------
import numpy as np

def barrett_reduce(x: np.ndarray, q: int) -> np.ndarray:
    """
    Barrett reduction for 0 <= x < 2^63 and ~48-bit q.
    Choose k so that 2^k > q (k=64 is fine).
    r = x - floor((x*mu)/2^k) * q, then correct to [0,q).
    """
    k = 64
    mu = (1 << k) // q                      # floor(2^k / q)

    xo = x.astype(object)                   # Python big-int arithmetic
    t  = ((xo * mu) >> k).astype(object)    # floor((x*mu)/2^k)
    r  = (xo - t * q).astype(int)           # r in [-(q-1), 2q-1]

    # Correct r into [0, q)
    r = np.where(r >= q, r - q, r)
    r = np.where(r <  0, r + q, r)

    return r.astype(np.int64)


In [12]:
# ------------------------------
# Testbench
# ------------------------------
def test_barrett_reduce():
    # Choose a modulus q (e.g., a 30-bit prime commonly used in FHE/RNS)
    q = qp[0]  # Example: 2^30 - 35, a common FHE-friendly prime

    # Generate random test values: 0 <= x < q^2
    np.random.seed(42)
    num_tests = 1000
    max_x = q * q - 1
    x_vals = np.random.randint(0, min(max_x, 2**63 - 1), size=num_tests, dtype=np.int64)

    # Compute expected results using built-in modulo
    expected = x_vals % q

    # Compute Barrett reduction
    actual = barrett_reduce(x_vals, q)

    # Compare
    if np.array_equal(actual, expected):
        print("✅ All tests passed!")
    else:
        # Find first mismatch
        mismatches = np.where(actual != expected)[0]
        if len(mismatches) > 0:
            i = mismatches[0]
            print(f"❌ Mismatch at index {i}:")
            print(f"  x = {x_vals[i]}")
            print(f"  Expected: {expected[i]}")
            print(f"  Got:      {actual[i]}")
            print(f"  Modulus q = {q}")
        assert False, "Barrett reduction failed!"

if __name__ == "__main__":
    test_barrett_reduce()

✅ All tests passed!


In [10]:
6908764162452635058%281474976317441

242343717363154

In [ ]:
# ------------------------------
# Uniform mod-q sampler
# ------------------------------
def sample_uniform_mod_q(n: int, q: int, use_barrett: bool = False,
                         rng: np.random.Generator | None = None) -> np.ndarray:
    """
    Uniform residues in [0, q). The hardware typically uses Barrett; for
    Python we can use x % q or enable barrett for parity with RTL.
    """
    if rng is None:
        rng = np.random.default_rng()
    # Draw 64-bit randoms (as if from a PRNG/TRNG) and reduce mod q
    raw = rng.integers(0, 1 << 63, size=n, dtype=np.int64)
    if use_barrett:
        return barrett_reduce(raw, q)
    else:
        return (raw % q).astype(np.int64)